In [1]:
from casstools.assignment import Assignment
Assignment().submit()

✅ TIMESTAMP  : 2024-01-06 19:10
💣 ERROR GETTING COURSE INFORMATION 💣
❌ Error Details: 404 Client Error: Not Found for url: https://cass.app.cent-su.org/courses/git-casstools
Possible Causes:
 - Is the course: 'git' a registered course?
 - Is the term: 'casstools' an term in that course?


In [2]:
filespec="/home/jovyan/library/ist256/spring2024/00-testing/testing.ipynb"
from casstools.assignment import Assignment
Assignment(filespec=filespec).submit()

✅ TIMESTAMP  : 2024-01-06 19:10
✅ COURSE     : ist256
✅ TERM       : spring2024
💣 ERROR GETTING YOUR USER INFORMATION 💣
❌ Error Details: 404 Client Error: Not Found for url: https://cass.app.cent-su.org/courses/ist256-spring2024/roster/myroles
Possible Causes:


NameError: name 'me' is not defined

In [1]:
from assignment import Assignment
Assignment()._confirm_submission(False,True)

ℹ Submit Again? [y/n] ❓  ds


False

In [3]:
from datetime import datetime

In [17]:
assignment_due_date = "2024-09-20 23:59"
due = datetime.strptime(assignment_due_date, "%Y-%m-%d %H:%M")
now = datetime.now()

In [15]:
now.strftime("%Y-%m-%d %H:%M")

'2024-01-06 12:34'

In [21]:
now <= due

True

In [2]:
import pandas as pd

In [12]:
pd.read_json("data/demo.json", lines=True, orient="values")

,name,age
0,abby,10
1,bob,11
2,chris,12


In [5]:
from assignment import Assignment
Assignment().read_submission_log()

,TIMESTAMP,COURSE,TERM,USER,STUDENT,PATH,ASSIGNMENT,POINTS,DUE DATE,LATE,STATUS,RECIEPT
0,2024-01-06 13:14:00,ist256,spring2024,mafudge@syr.edu,True,ist256/spring2024/00-testing/testing.ipynb,testing.ipynb,5,2024-09-20 23:59,False,On Time,28158a2cb4b3c23fc46b7a169b287c3c
1,2024-01-06 13:15:00,ist256,spring2024,mafudge@syr.edu,True,ist256/spring2024/00-testing/testing.ipynb,testing.ipynb,5,2024-09-20 23:59,False,On Time,28158a2cb4b3c23fc46b7a169b287c3c
